In [1]:
!python --version

Python 3.10.14


In [2]:
!pip install -r requirements.txt

  Using cached chromadb-0.5.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached google_generativeai-0.7.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached langchain-0.2.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_google_genai-1.0.7-py3-none-any.whl.metadata (3.8 kB)
  Using cached langchain_community-0.2.6-py3-none-any.whl.metadata (2.5 kB)
  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached pydantic-2.8.0-py3-none-any.whl.metadata (123 kB)
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl.metadata (6.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 260.0 kB/s eta 0:00:00a 0:00:01
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached op

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import Chroma
import google.generativeai as genai
from langchain.text_splitter import CharacterTextSplitter


In [29]:
with open('.env', 'w') as f:
    f.write('GOOGLE_API_KEY=api\n')

In [31]:
load_dotenv('.env')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [32]:
llm = ChatGoogleGenerativeAI(model="models/gemini-pro")

In [33]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [34]:
pdf_directory = "data/"
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=2500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

all_pages = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split(text_splitter)
    all_pages.extend(pages)

In [35]:
db = Chroma.from_documents(pages, embedding)
db

In [36]:
retriever = db.as_retriever(search_kwargs={"k": 7})

In [37]:
template = """
Вы - помощник с искусственным интеллектом.
Отвечайте, исходя из предоставленного контекста.

context: {context}
input: {input}
answer:
"""

In [38]:
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [41]:
response = retrieval_chain.invoke({"input": " Какие существуют методы испытаций и какие им соответствуют ГОСТы?"})

In [42]:
print(response["answer"])

**Методы испытаний** | **ГОСТы**
------- | --------
Испытание на растяжение | ГОСТ 10006
Испытание на твердость | ГОСТ 9012
Испытание на сплющивание | ГОСТ 8695
Гидравлическое испытание | ГОСТ 3845
Испытание на загиб | ГОСТ 3728
Испытание на раздачу | ГОСТ 8694
Испытание на бортование | ГОСТ 8693
